## Similarity search for lightweight semantic search options

######  We have tried Haystack but the resulta sre not very accurate, so changed to BM25 so thta rather than embedding we can use vectoer search directly from tokens.

## Bestmatch25 Trials
#### https://opensourceconnections.com/blog/2015/10/16/bm25-the-next-generation-of-lucene-relevation/
#### http://ethen8181.github.io/machine-learning/search/bm25_intro.html

In [1]:
import pandas as pd

df = pd.read_csv('City_data1.csv')
df['out'] = df['sights']+df['desc']
df.head()

,city,country,beach,mountain,history,food,city_life,countryside,nightlife,couple_friendly,outdoor,spiritual,sights,desc,Best_time,img,out
0,Paris,France,0,0,1,1,1,0,1,1,0,0,"Eiffel Tower,Louvre Museum,Cathdrale notre dam...","History, culture, food and fashion in one pict...",Summer (June to August): Summer is festival se...,https://img.delicious.com.au/DGZCHR1s/del/2018...,"Eiffel Tower,Louvre Museum,Cathdrale notre dam..."
1,New york,USA,0,0,0,1,1,0,1,1,0,0,"Central Park,The Museum of Modern Art,Statue o...","The tallest buildings, biggest museums, and be...",Spring (April / May) and Fall (September / Oct...,https://img.delicious.com.au/ttEaxwsa/del/2018...,"Central Park,The Museum of Modern Art,Statue o..."
2,Rome,Italy,1,0,1,1,1,1,1,1,1,1,"Colosseum,Pantheon,Trevi Fountain,St. Peter's ...",The eternal city is a heartland for food and h...,Late fall to early spring (October to April): ...,https://img.delicious.com.au/BTJu9b76/del/2018...,"Colosseum,Pantheon,Trevi Fountain,St. Peter's ..."
3,London,UK,1,1,1,1,1,0,1,1,0,0,"Tower of London,The National Gallery,Buckingha...",An essential destination for every traveller’s...,Summer (June to August): With average temps ar...,https://img.delicious.com.au/ZOmI5Coz/del/2018...,"Tower of London,The National Gallery,Buckingha..."
4,Tokyo,Japan,1,1,1,1,1,1,1,1,1,1,"Sensō-ji,Meiji Jingu,Tokyo Skytree,Shinjuku Gy...","Japan's capital with skyscrapers, museums, the...",October to February: Tokyo is a great destinat...,https://img.delicious.com.au/ErfAJ_zb/del/2018...,"Sensō-ji,Meiji Jingu,Tokyo Skytree,Shinjuku Gy..."


In [9]:
# !pip install rank_bm25
# !python -m spacy download en_core_web_sm
from rank_bm25 import BM25Okapi
import tqdm
import spacy
from rank_bm25 import BM25Okapi
from tqdm import tqdm


In [16]:
# !python -m spacy download en_core_web_sm
# nlp = spacy.load("en_core_web_sm")
import en_core_web_sm
# nlp = spacy.load('en_core_web_sm')
nlp = en_core_web_sm.load()

In [17]:
text_list = df.desc.str.lower().values
tok_text=[] # for our tokenised corpus
#Tokenising using SpaCy:
for doc in tqdm(nlp.pipe(text_list, disable=["tagger", "parser","ner"])):

    tok = [t.text for t in doc if t.is_alpha]
    tok_text.append(tok)

100it [00:00, 574.20it/s]


In [41]:
bm25 = BM25Okapi(tok_text)
def sim_search(query):
    tokenized_query = query.lower().split(" ")
    import time
    t0 = time.time()
    results = bm25.get_top_n(tokenized_query, df.desc.values, n=6)
    t1 = time.time()
    for i in results:
        print(df[df['desc']==i]['city'].iloc[0])
        print(i)

In [42]:
query4 = "nightlife and party with friends"
query2 = "outdoor activities, snowy mountain biking and guys stuff"
query = "romantic place to visit with my wife"
query5 = "i want to go to sunny, beach and crowd vibe. something like hollywood"
query1="temple mosque and spiritual place"
sim_search(query)

Agra
See the Taj Mahal up-close in Agra.North Indian city with Mughal Empire landmarks like the Taj Mahal mausoleum & red-walled Agra Fort.Where better to go for a romantic vacation than to the great testament of love, the Taj Mahal? Built by the grieving Mughal Emperor Shah Jahan in memory of his late wife Mumtaz Mahal, the Taj Mahal’s unrivaled beauty explains why it’s regarded as one of the eight wonders of the world. A visit to India wouldn’t be complete without it. Other must-experience destinations in Agra are two other UNESCO World Heritage Sites, the sandstone Red Fort and Fatehpur Sikri.
Seattle
There’s more to be experienced than coffee and grunge in this ever-evolving city.In Seattle you simply can’t skip the Central Public Library – a modern architectural marvel of glass grids, unusual shapes, and a “book spiral” that climbs four stories. Stroll over to Pike Place Market to visit the original Starbucks and play catch with a fishmonger. In the heart of the city lies Chihuly 

#### Better result than haystack
##### final model to be used in sim_search